## Importacion de librerias

In [1]:
import numpy as np
import pandas as pd
import warnings
import string
import gensim
from stop_words import get_stop_words
from itertools import chain
from joblib import Parallel, delayed
import gc
from gensim import corpora, models
import unidecode
import io
warnings.simplefilter("ignore")

C:\Users\lssro\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Carga de Archivos

- Archivo de lematizacion: raiz de palabras
- Archivo de transacciones
- Archivo metadatos tabla transacciones

Archivo lematizacion

In [2]:
f = io.open('lemmatization-es.txt', mode="r", encoding="utf-8")
r_lemmatization=f.readlines()

dic={}
for i in r_lemmatization:
    tmp=i.replace("\n","").split("\t")
    dic[tmp[1]]=tmp[0]

Transacciones

In [3]:
a=pd.read_csv('dt_trxpse_personas_2016_2018_muestra_adjt.csv', header= None,error_bad_lines=False,warn_bad_lines=False)
print (a.shape)

(11806137, 11)


In [4]:
a=a.rename(columns={0:"id_trn_ach",1:"id_cliente",2:"fecha",
                  3:"hora",4:"valor_trx",5:"ref1",6:"ref2",
                  7:"ref3",8:"sector",9:"subsector",10:"descripcion",})  # asignacion de encabezados

Metadatos Transacciones

In [5]:
meta_data=pd.read_excel('Metadatos_Tabla.xlsx')
meta_data

,name,type,comment
0,id_cliente,bigint,NaN
1,seg_str,string,NaN
2,ocupacion,string,NaN
3,tipo_vivienda,string,NaN
4,nivel_academico,string,NaN
5,estado_civil,string,NaN
6,genero,string,NaN
7,edad,int,NaN
8,ingreso_rango,string,NaN


### Muestra Dataset

In [6]:
a.head(10)

,id_trn_ach,id_cliente,fecha,hora,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion
0,230435642,3,20161207.0,113451,2122392.51,CC,NaN,NaN,\N,\N,\N
1,222356110,10,20161016.0,3424,148438.37,Referencia: Contrato: Valor:,CC,NaN,\N,\N,\N
2,309137749,10,20180120.0,195042,94025.19,CC,NaN,NaN,\N,\N,\N
3,324614737,10,20180326.0,192146,94430.07000000001,CC,NaN,NaN,\N,\N,\N
4,235344690,18,20170106.0,201317,670645.5699999999,MEDICINA PREPAGADA COLSANITAS,CE,NaN,\N,\N,\N
5,320049316,18,20180307.0,143513,706933.7,RECAUDO COLSANITAS,CC,NaN,\N,\N,\N
6,230519178,19,20161207.0,155840,306773.79,CC,NaN,NaN,\N,\N,\N
7,241307506,19,20170210.0,142809,701067.98,Pago de la Planilla Cesantias,CEDULA DE CIUDADANIA,NaN,SERVICIOS FINANCIEROS,OTROS SERVICIOS FINANCIEROS,Otras actividades auxiliares de las actividade...
8,316193109,24,20180220.0,153148,767571.01,CC,NaN,NaN,\N,\N,\N
9,282076350,26,20170914.0,70026,100277.45,Pago de la factura # CONJUNTO RESIDENCIAL PIET...,IDC,NaN,\N,\N,\N


### Filtrado de valores anomalos, con poca información o sin información

In [7]:
a=a[(a.valor_trx!="\\N")]
a=a[(a.ref1.isna()==False)|(a.ref2.isna()==False)|(a.ref3.isna()==False)|(a.descripcion!='\\N')|(a.sector!='\\N')|(a.subsector!='\\N')]
a=a.replace(np.nan,'')
a=a.replace('\\N','')

### Extracción del texto

extraemos la informacion contenida en las referencias (ref1,ref2,ref3) y se almacena en una sola columna

In [8]:
c= a.ref1.str.cat(a["ref2"],sep='-')
c= c.str.cat(a["ref3"],sep='-')

### Liberación de memoria

eliminamos las variables que ya no son útiles y llamamos la funcion gc.collect para liberar la memoria

In [9]:
del a #eliminar variable
gc.collect()# eliminar memoria que ocupaba la variable

63

### Definición de funciones para limpieza del texto

Función para remover puntuación

In [10]:
def remove_punctuation(text):
    # Replace punctuation with tokens so we can use them in our model
    text = text.replace('.', ' ')
    text = text.replace(',', ' ')
    text = text.replace('"', ' ')
    text = text.replace(';', ' ')
    text = text.replace('!', ' ')
    text = text.replace('?', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace('-', ' ')
    text = text.replace('_', ' ')
    text = text.replace('?', ' ')
    text = text.replace(':', ' ')
    text = text.replace("'", ' ')
    text = text.replace('#', ' ') 
    text = text.replace('$', ' ') 
    text = text.replace('%', ' ') 
    text = text.replace('/', ' ') 
    text = text.replace('*', ' ') 
    text = text.replace('+', ' ') 
    text = text.replace('<', ' ') 
    text = text.replace('>', ' ') 
    text = text.replace('=', ' ') 
    text = text.replace('@', ' ') 
    text = text.replace('[', ' ') 
    text = text.replace(']', ' ') 
    text = text.replace('^', ' ') 
    text = text.replace('`', ' ')
    text = text.replace('{', ' ') 
    text = text.replace('}', ' ') 
    text = text.replace('~', ' ')
    text = text.replace('|', ' ')
    return text

Función para eliminar signos de puntuación previamente definidos, convertir todos los caracteres a minúsculas y realizar una división de palabras.

In [11]:
def my_tokenizer(text):
    text = remove_punctuation(text)
    text = text.lower()  # downcase
    return text.split()

Función para eliminar acentuaciones y stop words

In [12]:
def preprocess(text):
    result = []
    for token in text:
        try:
            tmp=dic[token]
        except:
            tmp=token
        if tmp not in stop_words:
            tmp=unidecode.unidecode(tmp)
            result.append(tmp)
            
    return result

### Definición de stop words

Se importa libreria con la información de las stop words en el idioma español, se agregan a este listado palabras adicionales vistas en el conjunto de referencias y que carecen de sentido para el análisis.

Estas stop words serán eliminadas del texto.

In [13]:
stop_words = get_stop_words('es')
stop_words.append("pago")
stop_words.append("pse")
stop_words.append("referencia")
stop_words.append("ref")
stop_words.append("valor")
stop_words.append("fc")
stop_words.append("m")
stop_words.append("realizados")
stop_words.append("bog")
stop_words.append("agregado")
stop_words.append("numero")
stop_words.append("sc")
stop_words.append("idc")
stop_words.append("cr")
stop_words.append("d")
stop_words.append("cl")
stop_words.append("aa")
stop_words.append("sur")
stop_words.append("int")
stop_words.append("t")
stop_words.append("apoyo")
stop_words.append("saldar")
stop_words.append("pagar")
stop_words.append("sas")
stop_words.append("emcaliach")
stop_words.append("b")
stop_words.append("bogota")
stop_words.append("emcaliach")

### Eliminación de stop words

Se aplican las funciones de limpieza de texto por medio de apply

In [14]:
c= c.apply(my_tokenizer)

In [15]:
processed_docs=c.apply(preprocess)

Se libera memoria

In [16]:
del c
gc.collect()

0

### Búsqueda de registros por palabra

Se buscan los registros donde aparece una palabra determinada, con el fin de analizar con cuales otras palabras se relaciona.

In [17]:
k=0
for i in processed_docs:
    if "exito" in i:
        k=k+1
        if k==3:
            break

In [18]:
i #registro donde aparece la palabra

['traves', 'exito', 'cc']

### Listado de documentos preprocesados

In [19]:
processed_docs[:100]

0                                                   [cc]
1                           [referenciar, contratar, cc]
2                                                   [cc]
3                                                   [cc]
4                 [medicinar, prepagada, colsanitas, ce]
5                             [recaudar, colsanitas, cc]
6                                                   [cc]
7              [planilla, cesantias, cedula, ciudadania]
8                                                   [cc]
9      [facturar, conjuntar, residencial, pietra, san...
10     [facturar, conjuntar, residencial, pietra, san...
11                                  [plan, familiar, ce]
12                                  [plan, familiar, ce]
14                                                  [cc]
15                                                  [cc]
16                                                 [cpv]
17                  [irecaudo, claro, solucionar, movil]
18                    [facturar

### Construir diccionario con las palabras únicas de los documentos preprocesados

In [20]:
dictionary = gensim.corpora.Dictionary(processed_docs)

#### Muestra de las 10 primeras palabras del diccionario

In [21]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 cc
1 contratar
2 referenciar
3 ce
4 colsanitas
5 medicinar
6 prepagada
7 recaudar
8 cedula
9 cesantias
10 ciudadania


#### Número de documentos que son tenidos en cuenta para la construcción del diccionario

In [22]:
dictionary.num_docs

11568871

#### Filtrado del diccionario de palabras según análisis de frecuencias

Se eliminan las palabras que no se presentan en más de 15 documentos y aquellas que estan en todos los documentos quedando con un total de 13165 palabras.

In [23]:
dictionary.filter_extremes(no_below=15, no_above=1.0, keep_n=100000) 
# keep_n =100000 conserva sólo 100.000 de palabras en caso de que el diccionario tuviera mayor cantidad de palabras.

### Conteo de palabras por documento
Por cada documento se crea una lista de las palabras que se encuentran en este y se cuenta las veces que aparece esa palabra en el mismo documento

In [24]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

Vista del bag of words de un ejemplo de un documento preprocesado

In [25]:
bow_corpus[4310] #Ejemplo de documento

[(13, 1), (25, 1), (26, 1)]

In [26]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 13 ("facturar") appears 1 time.
Word 25 ("asociar") appears 1 time.
Word 26 ("bancoomevapfa") appears 1 time.


### Modelo Term frequency – Inverse document frequency (tfidf)

weight_{i,j} = wlocal(frequency_{i,j}) * wglobal(document\_freq_{i}, D)

Análisis de la frecuencia de la palabra en el documento vs los documentos donde aparece la misma palabra

Donde, 

TF = (Número de veces que la palabra clave aparece en el documento) / (Número total de palabras en el documento)

IDF = (Número total de documentos) / (Número total de veces que aparece la palabra en todos los documentos)

In [27]:
tfidf = models.TfidfModel(bow_corpus)

In [28]:
corpus_tfidf = tfidf[bow_corpus]

In [29]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 1.0)]


### Entrenamiento del modelo LDA
Topic Modeling and Latent Dirichlet Allocation

Topic Modeling es un tipo de modelado estadístico que permite descubrir los temas abstractos que aparecen en una colección de documentos. 
https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [30]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=7)

#### Tópicos hallados en los documentos
Se hallan las palabras que pertenecen a cada tema y su peso relativo, que explica el nivel de pertenencia de la palabra a cada tema.

In [31]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.636*"cc" + 0.059*"falabella" + 0.056*"payu" + 0.040*"transaccional" + 0.038*"portal" + 0.035*"exito" + 0.032*"tarjeta" + 0.031*"visar" + 0.025*"credito" + 0.019*"amex"
Topic: 1 Word: 0.338*"facturar" + 0.094*"cc" + 0.072*"suramericano" + 0.032*"tender" + 0.032*"comprar" + 0.031*"asociar" + 0.031*"bancoomevapfa" + 0.025*"virgin" + 0.025*"online" + 0.025*"polizas"
Topic: 2 Word: 0.101*"cmr" + 0.059*"servicio" + 0.054*"avianca" + 0.053*"eticket" + 0.051*"facturar" + 0.038*"comerciar" + 0.033*"bogota" + 0.033*"virtual" + 0.033*"camara" + 0.029*"cc"
Topic: 3 Word: 0.144*"postpago" + 0.137*"express" + 0.088*"facturar" + 0.045*"cc" + 0.031*"imponer" + 0.027*"tradicional" + 0.023*"vehiculos" + 0.019*"duplicar" + 0.019*"predial" + 0.016*"null"
Topic: 4 Word: 0.194*"referenciar" + 0.157*"cpv" + 0.128*"contratar" + 0.079*"express" + 0.074*"electronico" + 0.046*"comprar" + 0.042*"cc" + 0.026*"tiquete" + 0.015*"paquete" + 0.014*"prestamo"
Topic: 5 Word: 0.150*"traves" + 0.092*"cert

#### Se guarda el modelo entrenado

In [32]:
lda_model_tfidf.save('ldaref.model')

### Predicción nuevo documento

In [35]:
documento = 'Pago de factura telefonia movil'

In [36]:
my_tokenizer(documento)

['pago', 'de', 'factura', 'telefonia', 'movil']

In [37]:
preprocess(my_tokenizer(documento))

['facturar', 'telefonia', 'movil']

In [38]:
dictionary.doc2bow(preprocess(my_tokenizer(documento)))

[(13, 1), (23, 1), (5147, 1)]

In [40]:
for index, score in sorted(lda_model_tfidf[dictionary.doc2bow(preprocess(my_tokenizer(documento)))], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7749638557434082	 
Topic: 0.133*"tpni" + 0.109*"nit" + 0.052*"hogar" + 0.052*"multiplay" + 0.051*"obligacion" + 0.050*"facturar" + 0.029*"etb" + 0.028*"periodo" + 0.025*"contar" + 0.021*"credito"

Score: 0.025022879242897034	 
Topic: 0.338*"facturar" + 0.094*"cc" + 0.072*"suramericano" + 0.032*"tender" + 0.032*"comprar" + 0.031*"asociar" + 0.031*"bancoomevapfa" + 0.025*"virgin" + 0.025*"online" + 0.025*"polizas"

Score: 0.02500748820602894	 
Topic: 0.101*"cmr" + 0.059*"servicio" + 0.054*"avianca" + 0.053*"eticket" + 0.051*"facturar" + 0.038*"comerciar" + 0.033*"bogota" + 0.033*"virtual" + 0.033*"camara" + 0.029*"cc"

Score: 0.025005053728818893	 
Topic: 0.144*"postpago" + 0.137*"express" + 0.088*"facturar" + 0.045*"cc" + 0.031*"imponer" + 0.027*"tradicional" + 0.023*"vehiculos" + 0.019*"duplicar" + 0.019*"predial" + 0.016*"null"

Score: 0.02500056102871895	 
Topic: 0.236*"recargar" + 0.105*"nequi" + 0.038*"recaudar" + 0.034*"cc" + 0.024*"linear" + 0.021*"datar" + 0.021*"fligh

In [41]:
lda_model_tfidf[dictionary.doc2bow(preprocess(my_tokenizer(documento)))]

[(0, 0.025000002),
 (1, 0.025020648),
 (2, 0.025007486),
 (3, 0.025005052),
 (4, 0.025000002),
 (5, 0.025000002),
 (6, 0.77496606),
 (7, 0.025000561),
 (8, 0.025000175),
 (9, 0.025000002)]